In [9]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [10]:
train_image_directory = "/Users/annahauk/Desktop/NYBG/bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train"
validation_image_directory = "/Users/annahauk/Desktop/NYBG/bttai-nybg-2024/BTTAIxNYBG-validation/BTTAIxNYBG-validation"

In [11]:
filename_train = "/Users/annahauk/Desktop/NYBG/bttai-nybg-2024/BTTAIxNYBG-train.csv"
df_train = pd.read_csv(filename_train, header = 0)
filename_test = "/Users/annahauk/Desktop/NYBG/bttai-nybg-2024/BTTAIxNYBG-test.csv"
df_test = pd.read_csv(filename_test, header = 0)
filename_val = "/Users/annahauk/Desktop/NYBG/bttai-nybg-2024/BTTAIxNYBG-validation.csv"
df_val = pd.read_csv(filename_val, header = 0)

In [12]:
df_train.head(10)

,uniqueID,classLabel,classID,source,imageFile
0,2,occluded-specimens,8,L,a1a8b48e8cb142b3.jpg
1,3,microscope-slides,6,L,79599db2ac9092b6.jpg
2,4,illustrations-color,2,BHL,c449696f2f0d0d92.jpg
3,5,illustrations-color,2,P,80a8f4a393b4e08c.jpg
4,6,animal-specimens,0,AK,041a1c6e73313638.jpg
5,8,occluded-specimens,8,L,ccf1b1ccacb8f8b3.jpg
6,11,live-plants,4,US,246c8e9612111a24.jpg
7,12,biocultural-specimens,1,C,b0b337313164a0f0.jpg
8,13,microscope-slides,6,L,b3931bc4cc2b2925.jpg
9,15,illustrations-gray,3,BR,ebf7674e4c2c0e6e.jpg


In [14]:
# train a Small VGGNet using df_train, df_test, and df_val
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD

# load the VGG16 network, ensuring the head FC layer sets are left off
baseModel = VGG16(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dense(64, activation="relu")(headModel)
headModel = Dense(32, activation="relu")(headModel)
headModel = Dense(16, activation="relu")(headModel)
headModel = Dense(8, activation="relu")(headModel)
headModel = Dense(4, activation="relu")(headModel)
headModel = Dense(2, activation="relu")(headModel)
headModel = Dense(1, activation="sigmoid")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = SGD(lr=0.001)
model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random

# train the model using the training data
print("[INFO] training head...")
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(dataframe=df_train, directory=train_image_directory, x_col="filename", y_col="label", class_mode="binary", target_size=(224, 224), batch_size=32)

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(dataframe=df_val, directory=validation_image_directory, x_col="filename", y_col="label", class_mode="binary", target_size=(224, 224), batch_size=32)

model.fit(train_generator, validation_data=val_generator, epochs=10)

# evaluate the network
print("[INFO] evaluating network...")
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(dataframe=df_test, directory=train_image_directory, x_col="filename", y_col="label", class_mode="binary", target_size=(224, 224), batch_size=32)
print("[INFO] evaluating network...")
test_loss, test_acc = model.evaluate(test_generator)
print("test loss: {:.2f}".format(test_loss))
print("test accuracy: {:.2f}".format(test_acc))

# save the model to disk
print("[INFO] serializing network...")
model.save("bttai-nybg-2024.model", save_format="h5")

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)

In [ ]:
# plot the training loss and accuracy
import matplotlib.pyplot as plt
N = 10
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), model.history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), model.history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), model.history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), model.history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

model = Sequential()

# First CONV => RELU => POOL layer set
model.add(Conv2D(32, (3, 3), padding="same", input_shape=(64, 64, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Second CONV => RELU => POOL layer set
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Third CONV => RELU => POOL layer set
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# First (and only) set of FC => RELU layers
model.add(Flatten())
model.add(Dense(500))
model.add(Activation("relu"))

# Softmax classifier
model.add(Dense(classes=10))
model.add(Activation("softmax"))

TypeError: Dense.__init__() missing 1 required positional argument: 'units'

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [1]:
from sklearn.model_selection import train_test_split

X_test, y_test = train_test_split(df_train, test_size=0.2, random_state=42)


NameError: name 'df_train' is not defined

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=25)

In [ ]:
base_model = InceptionV3(weights='imagenet', include_top=False)